In [1]:
#Read the json FIle contain tweets and there labes 
#For train and test
from pyspark.sql import SparkSession
jobDir = "tweets1.json" # tain data
tweets = spark.read.json([jobDir])
tweets.count() #number of tweets in the file

6486

In [2]:
#select the object in json file
tweets = tweets.select("text", \
                     "Category" )

tweets.printSchema()

root
 |-- text: string (nullable = true)
 |-- Category: string (nullable = true)



In [3]:
from pyspark.ml.feature import RegexTokenizer,CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words")
#convert a collection of text documents to vectors of token counts. 
countVectors = CountVectorizer(inputCol="words", outputCol="features")
from pyspark.ml import Pipeline
from pyspark.ml.feature import  StringIndexer
#StringIndexer encodes a string column of labels to a column of label indices.
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

In [4]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=MulticlassClassificationEvaluator(), \
                    numFolds=5)
pipeline = Pipeline(stages=[regexTokenizer,label_stringIdx, countVectors,cv])
model2 = pipeline.fit(tweets) # use the train data to train the model

In [6]:
model2.save("vvvyes3")# save the model trained

In [7]:
#Data Ingestion and Extraction
from pyspark.sql import SparkSession
jobDir = "tweetsJson.json" # new data without label
test = spark.read.json([jobDir])
test.count() 

26

In [8]:
#select the object in json file
test = test.select("text")

test.printSchema()

root
 |-- text: string (nullable = true)



In [12]:
from pyspark.ml import PipelineModel
model2 = PipelineModel.load("vvvyes3")# use the model to classify the new data

In [13]:
predictions = model2.transform(test)# get the predictions of new data

In [16]:
selected = predictions.select("text",  "prediction").show(20,False) # display the predictions of new data

+-------------------------------------------------------------------------------------+----------+
|text                                                                                 |prediction|
+-------------------------------------------------------------------------------------+----------+
|كل يوم تاكد ان جود ام ف حيات اهم جودي شخصي ❤️                                        |1.0       |
|يع انتي ام مفتكرتيش انا ابقي ابن مين -اغ اتوبيس 2019                                 |1.0       |
| انا ام مات دهس والج يتمتع بحي هانء ماتوقف يوم واحد حرق قلوب الله يحرق قلب 💔😭      |0.0       |
|ام داز مقطع تقول حتا قرد رايح مسجد يسمع خطبه و نايم بيت                              |0.0       |
|ام مصور زي زقت طالع صوره 😭😭                                                        |0.0       |
|ام تصنع قلب بيو امل تصلح افسد حيا ترمرم خراب روح تسد ثقب جرح تمح الم صدر  تسعى…      |2.0       |
| ام                                                                                  |0.0       |
| ام ♥️       